In [2]:
import pandas as pd
import numpy as np
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api

print("loading language model")
# load language model (this takes a few minutes)
model = api.load('word2vec-google-news-300')
print("model loaded")

loading language model
model loaded


In [2]:
vectors = model.wv
del model
vectors.init_sims(True) # normalize the vectors (!), so we can use the dot product as similarity measure

print('embeddings loaded ')
print('loading docs ... ')

NameError: name 'model' is not defined

In [ ]:
keywords = ['housing', 'affordable', 'homelessness', 'accessory', 'dwelling', 'unit', 'ADU']

In [15]:
directory = '../data/legistar_corpus'
import os

documents = []
section_length = 10

for filename in os.listdir(directory): 
    with open (os.path.join(directory, filename)) as f: 
        lines = f.readlines()
        num_sections = len(lines) // section_length #TODO: get ceiling, not floor
        
        for i in range(num_sections): 
            document_segment = lines[i * section_length:(i+1)*section_length]
            documents.append(document_segment)

print (len(documents))
print (documents[0])
    
    

1268104
['City of Santa Clara\n', '\n', 'CANCELED\n', '\n', ' Planning Commission\n', '\n', 'Wednesday, July 11, 2018\n', '\n', '7:00 PM\n', '\n']
